In [24]:
# load JSON
import json
import pandas
 
data = pandas.read_json('airQualityDataframe-auber_NO.json')
data.tail()

,ds,y
58546,1531292400000,40.0
58547,1531314000000,26.0
58548,1531389600000,20.0
58549,1531411200000,33.0
58550,1531422000000,7.0


In [ ]:
import pickle

# open a file, where you stored the pickled data
file = open('auber/PM10_prophet.pkl', 'rb')

# dump information to that file
m = pickle.load(file)
print(type(m))
# close the file
file.close()

In [ ]:
# PROPHET validation
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days') # horizon: forecast ; initial: training (default: 3x horizon); 
df_cv.head()

In [ ]:

# Performance metrics
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

# mean squared error (MSE), root mean squared error (RMSE), mean absolute error (MAE), mean absolute percent error (MAPE)
# computed on a rolling window of the predictions in df_cv after sorting by horizon (ds minus cutoff).
# By default, 10% of the predictions in each window (can be changed) 
# high rollingwindow --> smoother average curve 

# Visualize cross-validation
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')
